In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://tracy-nondoubtable-rose.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://tracy-nondoubtable-rose.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [8]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學 (Ming Hsin University of Science and Technology)，簡稱明新科大，是一所位於台灣新竹縣新豐鄉的私立科技大學。

以下是明新科技大學的簡要介紹：

1.  **創校歷史與沿革：**
    *   創立於1966年，前身為「明新工業專科學校」。
    *   1997年改制為「明新技術學院」。
    *   2002年正式升格為「明新科技大學」。
    *   學校歷史悠久，擁有超過半世紀的辦學經驗。

2.  **地理位置優勢：**
    *   位於新竹，鄰近新竹科學園區，與台灣的科技產業鏈連結緊密。這使得學校在產學合作、實習機會和畢業生就業方面具有顯著優勢。

3.  **辦學特色與定位：**
    *   **應用型科技大學：** 學校定位為一所「產學導向」的科技大學，注重理論與實務結合，以培養具備實作技能、符合產業需求的人才為目標。
    *   **專業領域：** 主要發展方向涵蓋工程、管理、服務事業及人文社會等領域。設有工程學院、管理學院、服務事業學院及人文社會學院。
    *   **產學合作：** 強調與業界的緊密合作，透過產業實習、專題研究、技術研發等方式，讓學生在學期間就能接觸產業實務，提升職場競爭力。
    *   **就業導向：** 課程設計以市場需求為導向，致力於提升學生的專業證照取得率和畢業生就業率，在業界擁有良好的口碑。

4.  **校訓與精神：**
    *   秉持「勤勞樸實」的校訓，鼓勵學生踏實學習、努力不懈。

總體而言，明新科技大學是一所積極回應產業發展需求，為國家培育具備專業技能與實務能力的應用型人才的重要學府。


In [9]:
result2 = stateful_query("校長是誰？")
print(result2)

截至我所知的最新資訊 (通常更新到2023年底或2024年初)，明新科技大學的現任校長是 **劉國偉 教授**。

劉國偉教授於2023年2月1日接任明新科技大學校長一職。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 05:44:37] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U967a7ddada8eaab9e15d60d0f3029ae7","events":[]}
BODY:  {"destination":"U967a7ddada8eaab9e15d60d0f3029ae7","events":[{"type":"message","message":{"type":"text","id":"594029997753630996","quoteToken":"Uvf0sOf66IPyG4jREiE8imaywhnGlnJKIb1eF_NvU4LNo5nNKq_cRwa1lWD5G_Sa0B1NDaq2iXFgHPIRzwIO-nHIXQAqHRmUM5R5cJUWQfGou1aFcZbqg-d1jtPRRdGtn0MuKK73vjtUeZnL3EO24g","markAsReadToken":"NqJDiw2OgqnirAc_Bx1MD8z1uvL9gvCQPk2qeOzPiwbPBc3GxGvPn7HMzzlphL2HrpbNNsW5nB2LhbWBIj1-N9cDqlGbYgee4PbRX4Wew0vLj_47s2ZpawzQZ_AhBZ1VfGS9-Vr1A0P1VqmlifSdD8HV5KfkhG6y3BNY7PfmFZlS_2PtxMo4SulUNP_cDkQlQ5Tq2rerio087DEPqEW9ew","text":"AI 校長是誰"},"webhookEventId":"01KDHQWDQZE8VCQFT0SX9V4M1D","deliveryContext":{"isRedelivery":false},"timestamp":1766900774145,"source":{"type":"user","userId":"U63019324a610a0977db9bdaed8d8f469"},"replyToken":"1589e6639bc74c92ad373844938ed8a0","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 05:46:16] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 06:00:30] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U967a7ddada8eaab9e15d60d0f3029ae7","events":[]}
BODY:  {"destination":"U967a7ddada8eaab9e15d60d0f3029ae7","events":[{"type":"message","message":{"type":"text","id":"594031457925071361","quoteToken":"keYynCRJ_UnqxQ2Qw5x6ohzCtR2Qh4E20RGeoD6lbs9aX6Vir-XIK9kWFY1-fnUd0quELivPelHprWERr3Zzp0UmGFkr3mbXEwjm0DYCc_Z5uTi_RciefNb-nNHmBQLLwC1f8ugue8bnCly9VAytJw","markAsReadToken":"PosaD9CQkna_fjFo22CTH0NRSC5Pj29jl8u9ibWJ5GV5BNLnGQ8UY6c6advQwtJHnmOTATxlLeL-MQlSPzkcDg_oegDKBsMEICKi9Z8gwbWcqNEZP4wVf9ypsbTwSlYIImllRtkdZl3e9ffny4tnMXGwZfa6gO53nTXDlOMerWsNYnW4NFsg8ytmxEJYUmz5QNWjFwCqXtgEEMETFkkFJA","text":"AI 校長是誰"},"webhookEventId":"01KDHRPZE30XMX8P2KYS9WWRVQ","deliveryContext":{"isRedelivery":false},"timestamp":1766901644404,"source":{"type":"user","userId":"U63019324a610a0977db9bdaed8d8f469"},"replyToken":"9ef24956dea24196a69d22a7daef99fb","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 06:00:46] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U967a7ddada8eaab9e15d60d0f3029ae7","events":[{"type":"message","message":{"type":"text","id":"594031636938228053","quoteToken":"YyZi5qVT2jiyMSbQusm3zv2ftCxmstdsQuAKeGwt44Sar3E1QAzk25VjVO8LlbyGV-iR_Jy3S6seXnuL_gfuORA6TSBXv4hdFH29CCSkGnsidMhPsPFrMNJ1_GdW_eeK4_o9-2LDosfL15_6M5sA8g","markAsReadToken":"esfe-oJXmETLD6a6wgy0L8S10IeipUVP-G13bZs03g1GBstCxV7P3KafzvC-wbJmPPBNmfu1edOu9YRZ-s6oPkuS2HbraB9MdIdhpkXQVDaX-WbCLNSgAgu7Iupr-ypHS5e1Q5j9f84nDqYe3t7IF1aUDglIYmbZ3fIlb4YmnnFYYMTnhDTxYUrZHzgaO1b2Ke9Jdz4sDFTaJrPQl6J0uA","text":"AI 今天是禮拜幾"},"webhookEventId":"01KDHRT7RMSX2B9RDFSHVV8N0Q","deliveryContext":{"isRedelivery":false},"timestamp":1766901751062,"source":{"type":"user","userId":"U63019324a610a0977db9bdaed8d8f469"},"replyToken":"a4efa475b833473681cdfd228b180dff","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/Dec/2025 06:02:33] "POST / HTTP/1.1" 200 -
